In [1]:
import mxnet as mx
import ROOT as rt
import numpy as np
import math

import datetime

Welcome to JupyROOT 6.10/00


In [3]:
start=datetime.datetime.now()

jetset=[]
labels=[]
maxx=0.4
maxy=0.4                  
arnum=16
dx=0
bx=2*maxx/(2*arnum+1)
by=2*maxy/(2*arnum+1)
read=rt.TFile('jet2.root')
record=mx.recordio.MXIndexedRecordIO('tmpl.idx','tmpl.rec','w')
jet=rt.gDirectory.Get('jetAnalyser/jetAnalyser')
entries=jet.GetEntriesFast()
overlap=np.zeros(shape=(3,2*arnum+1,2*arnum+1))
gover=np.zeros(shape=(3,2*arnum+1,2*arnum+1))
qover=np.zeros(shape=(3,2*arnum+1,2*arnum+1))
for ent in range(entries):
    #if ent>1000:
    #    break
    jet.GetEntry(ent)
    #if jet.pt<300 or jet.pt>400:
        #continue
    if jet.partonId==0:
        continue
    palet=np.zeros(shape=(3,2*arnum+1,2*arnum+1),dtype=np.float16)
    r=0
    g=0
    b=0
    for i in range(len(jet.dau_pt)):
        x=int(math.floor((jet.dau_deta[i]/bx)+0.5)+arnum)
        if x<0 or x>2*arnum:
            continue
        y=int(math.floor((jet.dau_dphi[i]/by)+0.5)+arnum)
        if y<0 or y>2*arnum:
            continue
        pt=jet.dau_pt[i]
        if jet.dau_charge[i]==0:
            palet[1][x][y]+=pt

            if palet[1][x][y]>g:
                g=palet[1][x][y]
        else:
            palet[0][x][y]+=pt
            palet[2][x][y]+=1

            if palet[0][x][y]>r:
                r=palet[0][x][y]
            if palet[2][x][y]>b:
                b=palet[2][x][y]
            #pass
    #record.write(palet)
    for i in range(2*arnum+1):
        for j in range(2*arnum+1):
            if r!=0:
                palet[0][i][j]=255*palet[0][i][j]/r
            if g!=0:
                palet[1][i][j]=255*palet[1][i][j]/g
            if b!=0:
                palet[2][i][j]=255*palet[2][i][j]/b
    
    jetset.append(palet)
    if (ent%1000==0):
        jetset=np.array(jetset,dtype=np.uint8)
        for l in range(len(jetset)):
            record.write_idx(dx,str(jetset[l].tolist()))
            dx+=1
        jetset=[]
    if jet.partonId==21:
        labels.append(1)
        gover=np.add(gover,palet)
    else:
        labels.append(0)
        qover=np.add(qover,palet)
    overlap=np.add(overlap,palet)
    
    
jetset=np.array(jetset,dtype=np.uint8)
for l in range(len(jetset)):
    record.write_idx(dx,str(jetset[l].tolist()))
    dx+=1
[r,g,b]=[0,0,0]
[gr,gg,gb]=[0,0,0]
[qr,qg,qb]=[0,0,0]
for i in range(2*arnum+1):
        for j in range(2*arnum+1):
            if overlap[0][i][j]>r:
                r=overlap[0][i][j]
            if overlap[1][i][j]>g:
                g=overlap[1][i][j]
            if overlap[2][i][j]>b:
                b=overlap[2][i][j]
                
for i in range(2*arnum+1):
        for j in range(2*arnum+1):
            if r!=0:
                overlap[0][i][j]=overlap[0][i][j]/r
            if g!=0:
                overlap[1][i][j]=overlap[1][i][j]/g
            if b!=0:
                overlap[2][i][j]=overlap[2][i][j]/b
record.close()
labels=np.array(labels)
jetset=np.array(jetset,dtype=np.uint8)
print jetset.shape
data=jetset.reshape((len(jetset),3*33*33))
print datetime.datetime.now()-start

(47, 3, 33, 33)
0:04:14.898140


In [10]:
len(palet.tolist()[1])

33

In [ ]:
0:04:22.827886
        0:03:52.983626
start=datetime.datetime.now()
record=mx.recordio.MXIndexedRecordIO('tmpl.idx','tmpl.rec','w')
j=0
for i in data[:]:
    record.write_idx(j,str(i.tolist()))
    j+=1
record.close()

print datetime.datetime.now()-start

In [19]:
np.uint8(jetset[1][1][16][:])

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,  12,   0, 255,   0,   0,   0,   0,   0,   2,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0], dtype=uint8)

In [53]:
start=datetime.datetime.now()
for i in data:
    if(eval(str(tuple(i)))!=tuple(i)):
        print "b"
        break
print datetime.datetime.now()-start


0:00:05.532844


In [52]:
start=datetime.datetime.now()
j=0
for i in data:
    if(eval(str(i.tolist()))!=i.tolist()):
        print j
        break
    j+=1
print datetime.datetime.now()-start


0:00:03.899646


In [178]:
start=datetime.datetime.now()
record=mx.recordio.MXRecordIO('tmpt.rec','w')
for i in data[:]:
    record.write(str(tuple(i)))
record.close()
print datetime.datetime.now()-start

0:00:47.633305


In [184]:
start=datetime.datetime.now()
record=mx.recordio.MXRecordIO('tmpt.rec','r')
j=0
while True:
    item = record.read()
    if not item:
        break
    if(eval(item)!=tuple(data[j])):
        print j
        break
    j+=1
record.close()
print j,datetime.datetime.now()-start

14948 0:01:08.726526


In [21]:
start=datetime.datetime.now()
record=mx.recordio.MXIndexedRecordIO('tmpl.idx','tmpl.rec','w')
j=0
for i in data[:]:
    record.write_idx(j,str(i.tolist()))
    j+=1
record.close()

print datetime.datetime.now()-start

0:00:03.013569


In [183]:
start=datetime.datetime.now()
record=mx.recordio.MXRecordIO('tmpl.rec','r')
j=0
while True:
    item = record.read()
    if not item:
        break
    if(eval(item)!=data[j].tolist()):
        print j
        break
    j+=1
record.close()
print j,datetime.datetime.now()-start

14948 0:01:06.879232


In [ ]:
start=datetime.datetime.now()
record=mx.recordio.MXRecordIO('tmpl.rec','w')
for i in len(labels):
    header=mx.recordio.IRHeader(0,labels[i],123,0)
    record.write(str(data[i].tolist()))
record.close()
print datetime.datetime.now()-start

In [ ]:
header=mx.recordio.IRHeader(0,labels[1],123,0)
packed=mx.recordio.pack_img(header,np.array([[[23,27,45],[28,32,50],[36,40,59],[35,39,58]],[[91,92,113],[97,98,119],[168,169,167],[166,167,165]]]))

In [15]:
np.array([[[23,27,45],[28,32,50],[36,40,59],[35,39,58]],[[91,92,113],[97,98,119],[168,169,167],[166,167,165]]])

array([[[ 23,  27,  45],
        [ 28,  32,  50],
        [ 36,  40,  59],
        [ 35,  39,  58]],

       [[ 91,  92, 113],
        [ 97,  98, 119],
        [168, 169, 167],
        [166, 167, 165]]])

In [26]:
import os
os.system("ls -alh")

0

total 416M
drwxr-xr-x 4 root root 4.0K Jun 29 07:04 .
drwxr-xr-x 1 root root 4.0K Jun 21 06:06 ..
drwxr-xr-x 8 root root 4.0K Jun 21 06:05 .git
drwxr-xr-x 2 root root 4.0K Jun 21 06:27 .ipynb_checkpoints
-rw-r--r-- 1 root root  11K Jun 29 07:04 datamake.ipynb
-rw-r--r-- 1 root root 100K Jun 21 06:05 jet run-Copy1.ipynb
-rwxr-xr-x 1 root root  24M May 31 07:15 jet2.root
-rw-r--r-- 1 root root 173K Jun 21 06:05 jetrun-Copy1.ipynb
-rw-r--r-- 1 root root 200K Jun 21 06:05 jetrun-Copy2.ipynb
-rw-r--r-- 1 root root 122K Jun 22 06:27 jetrun.ipynb
-rw-r--r-- 1 root root  14K Jun 21 06:05 mnist_python-Copy1.ipynb
-rwxr-xr-x 1 root root  20K Jun 21 06:05 mnist_python.ipynb
-rw-r--r-- 1 root root  16M Jun 22 07:03 tmp.rec
-rw-r--r-- 1 root root 212K Jun 29 05:35 tmpl.idx
-rw-r--r-- 1 root root 141M Jun 29 05:35 tmpl.rec
-rw-r--r-- 1 root root 235M Jun 27 08:28 tmpt.rec
